In [1]:
import nltk
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import wordnet
import string
from nltk.corpus import stopwords
import numpy as np
# DOC2VEC
import math as m
from gensim.models.doc2vec import Doc2Vec,TaggedDocument

C:\Users\Dell\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [39]:
wl=WordNetLemmatizer()


def get_tag(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.wordnet.ADV
    else:
        return wordnet.wordnet.NOUN
    
def get_lem(text):
    t=""
    for m in text:
        t+=wl.lemmatize(m[0].lower(),get_tag(m[1]))+" "
    return t.strip()

def get_vecs(data,max_epochs,vec_size):
    
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data[0]+data[1])]
    
    alpha = 0.025
    model = Doc2Vec(size=vec_size,
                    alpha=alpha, 
                    min_alpha=0.00025,
                    min_count=1,
                    dm =1)
    model.build_vocab(tagged_data)

    for epoch in range(max_epochs):
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.epochs)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha
    
#     vec=[]

#     for i in data[0]:
#         vec.append(model.infer_vector(word_tokenize(i)))
    
#     quer=model.infer_vector(word_tokenize(data[1][0]))
#     del model
#     print(quer.shape)
#     print(model.docvecs[-1].shape)
    return model.docvecs
    return vec,quer
    
    

In [46]:
train_file="./data.tsv"
file=open(train_file,"r",encoding="utf-8")
num=["1","2",'3','4','5','6','7','8','9','0']
labels=[]

def batch_nlp(batch_size=1000,vec_size=200,max_epochs=10,nu=2,mode=1,file=file):
    batch_size=(batch_size//10)*10
    m=0
    k=0
    query_f=[]
    query_ids=[]
    passage_f=[]
    dataset=[]
    labels=[]
    z=0
    zero=nu
    for f in file:
        token=f.strip().split("\t")
        query_id,query,passage,label=token[0],token[1],token[2],token[3]
        
        z+=1
        if z==10:
          z=0
          zero=nu
        if ((label=='0') and (zero == 0) and (mode==1)):
           continue
                     
        if label=='1':
          zero+=1
        zero-=1
        m+=1
        #Query Preprocessing
        query_ids.append(query_id)
        if k==0:
            for i in string.punctuation:
                query=query.replace(i,"")
            for i in num:
                query=query.replace(i,"")
            query=pos_tag(word_tokenize(query.strip()))
            query=get_lem(query)
            
            query_f.append(query)
            
            #Passage Preprocessing
        for i in string.punctuation:
            passage=passage.replace(i,"")
        for i in num:
            passage=passage.replace(i,"")
        passage=pos_tag(word_tokenize(passage.strip()))
        passage=get_lem(passage)
        passage_f.append(passage)    
        labels.append(float(label))
        k+=1
        if k==10:
            k=0
            
            vec=get_vecs((passage_f,query_f),max_epochs,vec_size)
#             print(q.shape)
            del passage_f
            del query_f
            passage_f=[]
            query_f=[]
            for i in range(10):
                dataset.append((vec[-1],vec[i]))

        if m==batch_size:
            m=0
            
            yield(np.array(dataset),np.array(labels),query_ids)
            del dataset
            del labels
            del query_ids
            labels=[]
            query_ids=[]
            dataset=[]
            
    yield(np.array(dataset),np.array(labels),query_ids)
        




In [47]:
def gives_mod(a):
    return ((a**2).sum())**(0.5)

def score(a,b):
    return abs(m.acos(a.dot(b)/(gives_mod(a)*gives_mod(b)))*(180/m.pi))/360    

In [48]:
# n=1000000
batch_size=10
max_epochs=50
vec_size=256

In [49]:
# data,label,q=next(batch_nlp(100,vec_size,max_epochs,nu=9,mode=1))

In [50]:
def acc(data,label):
    scor=[]
    for i in data:
        scor.append(score(i[0],i[1]))
    scor=np.array(scor)
    return((np.argmax(scor)==np.argmax(np.array(label))).sum()/scor.shape[0])*10

In [51]:
np.argmax(np.array(label)),label

(6, array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [52]:
data.shape

(100, 2, 256)

In [53]:
a=data[5][0]
b=data[5][1]
score(a,b)#0

0.16877709269724986

In [54]:
EvaluationFileName="eval2_unlabelled.tsv"
fw = open("answer.tsv","w",encoding="utf-8")
i=0
for batch in batch_nlp(10,vec_size,max_epochs,nu=9,mode=0,file=EvaluationFileName):
    logits=[]
    for a,b in batch[0]:
        logits.append(score(a,b))
    logits=[str(l[0]) for l in logits]
    logits="\t".join(logits)
    fw.write(batch[2][0]+"\t"+logits+"\n")
    print(i,end=" ")
    i+=1

IndexError: list index out of range